In [13]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import xml.etree.ElementTree as ET
import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/adrian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/adrian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
def limpiar_texto(texto):
    # Convertir el texto a minúsculas
    texto = texto.lower()
    
    # Eliminar puntuación
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    
    # Tokenizar el texto en palabras
    palabras = word_tokenize(texto)
    
    # Eliminar palabras vacías (stop words)
    palabras_limpias = [palabra for palabra in palabras if palabra not in stopwords.words('spanish')]
    
    # Unir las palabras limpias en una sola cadena de texto
    texto_limpio = ' '.join(palabras_limpias)
    
    return texto_limpio

In [15]:
import xml.etree.ElementTree as ET
import pandas as pd

# Parse the XML file
tree = ET.parse('dataset/train/TREC-2017-LiveQA-Medical-Train-1.xml')
root = tree.getroot()
# Crear una lista para almacenar las filas del CSV
data = []

# Iterar a través de la estructura XML y extraer la información
for question in root.findall('NLM-QUESTION'):
    questionid = question.get('questionid')
    fRef = question.get('fRef')
    subject = question.find('SUBJECT').text if question.find('SUBJECT') is not None else ''
    message = question.find('MESSAGE').text
    message_clean = limpiar_texto(message)
    
    for subquestion in question.find('SUB-QUESTIONS').findall('SUB-QUESTION'):
        subquestionid = subquestion.get('subqid')
        subquestion_focus = subquestion.find('ANNOTATIONS').find('FOCUS').text
        subquestion_type = subquestion.find('ANNOTATIONS').find('TYPE').text
        
        for answer in subquestion.find('ANSWERS').findall('ANSWER'):
            answerid = answer.get('answerid')
            answer_text = answer.text
            
            # Añadir la fila a la lista de datos
            data.append([ message_clean, subquestion_focus, subquestion_type, answer_text])

# Convertir la lista a un DataFrame de Pandas
df = pd.DataFrame(data, columns=['message', 'subquestion_focus', 'subquestion_type', 'answer'])

# Guardar el DataFrame a un archivo CSV
df.to_csv('dataset/train/train1.csv', index=False)

In [16]:
tree2 = ET.parse('dataset/train/TREC-2017-LiveQA-Medical-Train-2.xml')
root2 = tree2.getroot()

# Crear una lista para almacenar las filas del CSV
data2 = []

for question in root2.findall('NLM-QUESTION'):
    qid = question.get('qid')
    subject = question.find('SUBJECT').text if question.find('SUBJECT') is not None else ''
    message = question.find('MESSAGE').text
    message_clean = limpiar_texto(message)
    
    for subquestion in question.find('SUB-QUESTIONS').findall('SUB-QUESTION'):
        subquestion_focus = subquestion.find('ANNOTATIONS').find('FOCUS').text
        subquestion_type = subquestion.find('ANNOTATIONS').find('TYPE').text
        
        for answer in subquestion.find('ANSWERS').findall('ANSWER'):
            answer_text = answer.text
            
            # Añadir la fila a la lista de datos
            data2.append([message_clean, subquestion_focus, subquestion_type, answer_text])

# Convertir la lista a un DataFrame de Pandas
df = pd.DataFrame(data2, columns=['message', 'subquestion_focus', 'subquestion_type', 'answer'])

# Guardar el DataFrame a un archivo CSV
df.to_csv('dataset/train/train2.csv', index=False)

In [17]:
treeTest = ET.parse('dataset/train/TREC-2017-LiveQA-Medical-Train-2.xml')
rootTest = treeTest.getroot()

# Crear una lista para almacenar las filas del CSV
dataTest = []

# Iterar a través de la estructura XML y extraer la información
for question in rootTest.findall('NLM-QUESTION'):
    message = question.find('MESSAGE').text if question.find('MESSAGE') is not None else ''
    
    for subquestion in question.find('SUB-QUESTIONS').findall('SUB-QUESTION'):
        subquestion_focus = subquestion.find('ANNOTATIONS').find('FOCUS').text if subquestion.find('ANNOTATIONS') is not None and subquestion.find('ANNOTATIONS').find('FOCUS') is not None else ''
        subquestion_type = subquestion.find('ANNOTATIONS').find('TYPE').text if subquestion.find('ANNOTATIONS') is not None and subquestion.find('ANNOTATIONS').find('TYPE') is not None else ''
        
        for answer in subquestion.find('ANSWERS').findall('ANSWER'):
            answer_text = answer.text
            
            # Añadir la fila a la lista de datos
            data.append([message, subquestion_focus, subquestion_type, answer_text])

# Convertir la lista a un DataFrame de Pandas
df = pd.DataFrame(data, columns=['message', 'subquestion_focus', 'subquestion_type', 'answer'])

# Guardar el DataFrame a un archivo CSV
df.to_csv('dataset/test/test.csv', index=False)

In [22]:
train1 = pd.read_csv("dataset/train/train1.csv")
train2 = pd.read_csv("dataset/train/train2.csv")

# Unir los DataFrames
train = pd.concat([train1, train2], ignore_index=True)

# Guardar el DataFrame combinado a un nuevo archivo CSV
train.to_csv("dataset/train/train.csv", index=False)